# Исследование надёжности заёмщиков

**Цель исследования** 

Проверить:

1. есть ли зависимость между наличием детей и возвратом кредита в срок,

2. есть ли зависимость между семейным положением и возвратом кредита в срок,

3. есть ли зависимость между уровнем дохода и возвратом кредита в срок,

4. как разные цели кредита влияют на его возврат в срок.

**Входные данные** 

Данные кредитного отдела банка.

**Заказчик исследования**

Кредитный отдел банка. 

**Практическое применение результатов исследования**

Результаты исследования будут учтены при построении модели *кредитного скоринга* — специальной системы, которая оценивает способность потенциального заёмщика вернуть кредит банку.

## Обзор данных

Импортируем библиотеки для проекта.

In [1]:
import pandas as pd
from nltk.corpus import stopwords
import matplotlib.pyplot as plt
from pymystem3 import Mystem

mystem = Mystem()

Прочитаем данные и сохраним в переменную df.

In [2]:
df = pd.read_csv('data.csv') 

Посмотрим на первые 5 строк датафрейма df.

In [3]:
df.head()

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу


Получим общую информацию о данных в нашем датафрейме.

In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21525 non-null  int64  
 1   days_employed     19351 non-null  float64
 2   dob_years         21525 non-null  int64  
 3   education         21525 non-null  object 
 4   education_id      21525 non-null  int64  
 5   family_status     21525 non-null  object 
 6   family_status_id  21525 non-null  int64  
 7   gender            21525 non-null  object 
 8   income_type       21525 non-null  object 
 9   debt              21525 non-null  int64  
 10  total_income      19351 non-null  float64
 11  purpose           21525 non-null  object 
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


Каждая строка датафрейма содержит информацию об определенном физическом лице, запрашивающем кредит на известную цель (будем называть его заемщиком). Из таблицы можем видеть количество детей, трудовой стаж в днях, возраст, образование, семейное положение, пол, тип занятости, есть ли задолжность по кредиту, цель получения кредита и доход. 

<a id='describe'></a>  Посмотрим на уровень разброса значений в исследуемых данных. Для этого применим метод describe().

In [5]:
df.describe()

,children,days_employed,dob_years,education_id,family_status_id,debt,total_income
count,21525.000000,19351.000000,21525.000000,21525.000000,21525.000000,21525.000000,1.935100e+04
mean,0.538908,63046.497661,43.293380,0.817236,0.972544,0.080883,1.674223e+05
std,1.381587,140827.311974,12.574584,0.548138,1.420324,0.272661,1.029716e+05
min,-1.000000,-18388.949901,0.000000,0.000000,0.000000,0.000000,2.066726e+04
25%,0.000000,-2747.423625,33.000000,1.000000,0.000000,0.000000,1.030532e+05
50%,0.000000,-1203.369529,42.000000,1.000000,0.000000,0.000000,1.450179e+05
75%,1.000000,-291.095954,53.000000,1.000000,1.000000,0.000000,2.034351e+05
max,20.000000,401755.400475,75.000000,4.000000,4.000000,1.000000,2.265604e+06


<a id='intro'></a>
Рассмотрим уникальные значения данных для остальных столбцов (пол, образование, семейное положение, тип занятости, цель):

In [6]:
for column_name in df.columns:
        if column_name not in df.describe().columns:
            print(df[column_name].value_counts(), '\n')

среднее                13750
высшее                  4718
СРЕДНЕЕ                  772
Среднее                  711
неоконченное высшее      668
ВЫСШЕЕ                   274
Высшее                   268
начальное                250
Неоконченное высшее       47
НЕОКОНЧЕННОЕ ВЫСШЕЕ       29
НАЧАЛЬНОЕ                 17
Начальное                 15
ученая степень             4
Ученая степень             1
УЧЕНАЯ СТЕПЕНЬ             1
Name: education, dtype: int64 

женат / замужем          12380
гражданский брак          4177
Не женат / не замужем     2813
в разводе                 1195
вдовец / вдова             960
Name: family_status, dtype: int64 

F      14236
M       7288
XNA        1
Name: gender, dtype: int64 

сотрудник          11119
компаньон           5085
пенсионер           3856
госслужащий         1459
безработный            2
предприниматель        2
студент                1
в декрете              1
Name: income_type, dtype: int64 

свадьба                                 

Посмотрим на количество явных дубликатов в датафрейме с помощью цепочки методов duplicated().sum().

In [7]:
f'Количество явных дубликатов в таблице: {df.duplicated().sum()}'

'Количество явных дубликатов в таблице: 54'

<a id='null_values'></a> Получим процент пропущенных данных в столбце `total_income`. 

В столбце трудового стажа `days_employed` такое же количество пропущенных значений, как мы увидели из общей информации о датафрейме. 

In [8]:
f'Процент пропущенных значений в столбце total_income: {len(df[df["total_income"].isnull()])/df.shape[0]:.2%}'


'Процент пропущенных значений в столбце total_income: 10.10%'

In [9]:
f'Процент пропущенных значений в столбце days_employed: {len(df[df["days_employed"].isnull()])/df.shape[0]:.2%}'

'Процент пропущенных значений в столбце days_employed: 10.10%'

### Вывод

<div style="border:solid green 2px; padding: 20px">
    
В датафрейме 12 столбцов. Общее количество наблюдений в данных 21525. 

Каждая строка содержит информацию об определенном заемщике.

В таблице хранится следующая информация о заемщике: 
    
* количество детей в семье (children), 
* трудовой стаж в днях (days_employed), 
* возраст в годах (dob_years), 
* уровень образования (education), 
* индентификатор образования (education_id), 
* семейное положение (family_status), 
* идентификатор семейного положения (family_status_id), 
* пол (gender), 
* тип занятости (income_type), 
* имел ли задолжность по кредиту (debt), 
* ежемесячный доход (total_income) и 
* цель получения кредита (purpose). 
    
В ходе предварительного анализа данных выявлены следующие проблемы:
    
1. В столбце `children` есть артефакты: -1 ребенок и 20 детей.

2. В столбце `days_employed` пропущено 10.10% данных, а также есть отрицательные значения и тип данных число с плавающей точкой требует преобразования в integer. 
    
3. В столбце `dob_years` есть возраст со значением 0. 
    
4. В столбце `family_status` есть значение "Не женат / не замужем", которое требуется привести к строчному регистру по аналогии со всеми остальными.
   
5. В столбце `gender` есть артефакт: XNA. (Всего одна строка, поэтому можно просто удалить эту строчку.)  
    
6. В столбце `total_income` пропущено 10.10% данных, а также тип данных число с плавающей точкой требует преобразования в целочисленный. 
    
7. В столбце `education` в значениях есть неявные дубликаты. Требуется привести все значения к нижнему регистру. 

8. В данных выявлено 54 явных дубликата. 

Устраним проблемы в разделах ниже.

## Предобработка данных

### Обработка артефактов


#### Cтолбец children

Смотрим на столбец `children` (количество детей), запросим относительную частоту значений.

In [10]:
display(df['children'].value_counts(normalize=True))

 0     0.657329
 1     0.223833
 2     0.095470
 3     0.015331
 20    0.003531
-1     0.002184
 4     0.001905
 5     0.000418
Name: children, dtype: float64

0.2% заемщиков имеют -1 ребенок. Возможно, это означает утрату ребенка: заменим -1 на 0. 

0.3% заемщиков имеют 20 детей. Это маленький процент. Будем считать, что это "многодетные".

In [11]:
df['children'] = df['children'].replace({-1: 0})

Проверяем, как отработал метод replace:

In [12]:
f"Количество заемщиков с количеством детей -1: {len(df[df['children']== -1])}"

'Количество заемщиков с количеством детей -1: 0'

#### Cтолбец dob_years

[Ранее](#describe) в значениях `dob_years` мы заметили значение 0. 

Заменим 0 на среднее арифметическое.

In [13]:
df = df.replace({'dob_years': 0}, df['dob_years'].mean())

Проверяем, успешна ли замена 0-значений средним арифметическим.

In [14]:
df[df['dob_years'] == 0].count()

children            0
days_employed       0
dob_years           0
education           0
education_id        0
family_status       0
family_status_id    0
gender              0
income_type         0
debt                0
total_income        0
purpose             0
dtype: int64

#### Cтолбец days_employed

In [15]:
f"Количество заемщиков с положительным стажем: {len(df[df['days_employed'] > 0])}"

'Количество заемщиков с положительным стажем: 3445'

In [16]:
f"Количество заемщиков с отрицательным стажем: {len(df[df['days_employed'] < 0])}"

'Количество заемщиков с отрицательным стажем: 15906'

Посмотрим, как распределяются отрицательные и положительные значения в зависимости от типа занятости.

In [17]:
print('Положительный стаж имеют: \n')
df[df['days_employed'] > 0].groupby('income_type')['days_employed'].count()

Положительный стаж имеют: 



income_type
безработный       2
пенсионер      3443
Name: days_employed, dtype: int64

In [18]:
print('Отрицательный стаж имеют: \n')
df[df['days_employed'] < 0].groupby('income_type')['days_employed'].count()

Отрицательный стаж имеют: 



income_type
в декрете              1
госслужащий         1312
компаньон           4577
предприниматель        1
сотрудник          10014
студент                1
Name: days_employed, dtype: int64

Положительный стаж имеют все пенсионеры. 

Похоже, отрицательный стаж - это техническая ошибка и его можно считать положительным. 

Вычислим модуль числа с помощью встроенной функции abs и исправим отрицательный стаж на положительный.

In [19]:
print('Количество отрицательных значений до обработки:', df[df['days_employed'] < 0]['days_employed'].count())
df['days_employed'] = abs(df['days_employed'])
print('Количество отрицательных значений после обработки:', df[df['days_employed'] < 0]['days_employed'].count())

Количество отрицательных значений до обработки: 15906
Количество отрицательных значений после обработки: 0


Для удобства переведем значения из дней - в года и добавим такой столбец в исходный датафрейм.

In [20]:
df['years_employed'] = df['days_employed']/365

Посмотрим еще раз на описательную статистику стажа, но теперь в годах.

In [21]:
df['years_employed'].describe()

count    19351.000000
mean       183.328024
std        380.906522
min          0.066141
25%          2.539751
50%          6.011563
75%         15.172281
max       1100.699727
Name: years_employed, dtype: float64

Мы наблюдаем явные выбросы - аномально высокий стаж в 1100 лет, среднее - 183 года, большой разброс в данных с высоким стандартным отклонением. 

Посмотрим на раcпределение значений трудового стажа (в годах) по типу занятости. Разобьем значения стажа на 3 корзины и выведем количество клиентов в долях.

In [22]:
round(df.groupby('income_type')['years_employed'].value_counts(bins=2, normalize=True), 2)

income_type      years_employed             
безработный      (924.565, 1003.873]            0.50
                 (1003.873, 1083.021]           0.50
в декрете        (9.022, 9.032]                 1.00
                 (9.032, 9.041]                 0.00
госслужащий      (0.0669, 20.867]               0.81
                 (20.867, 41.625]               0.08
компаньон        (0.0335, 24.172]               0.88
                 (24.172, 48.262]               0.02
пенсионер        (900.426, 1000.663]            0.45
                 (1000.663, 1100.7]             0.45
предприниматель  (1.425, 1.427]                 0.50
                 (1.427, 1.428]                 0.00
сотрудник        (0.0148, 25.223]               0.88
                 (25.223, 50.381]               0.02
студент          (1.5830000000000002, 1.586]    1.00
                 (1.586, 1.587]                 0.00
Name: years_employed, dtype: float64

Среди пенсионеров и безработных наблюдается аномально высокий стаж - в пределах от 900 до 1100 лет.

Проверим, возможно, по какой-то причине этот стаж указан в других единицах - в часах. 
Вычислим стаж в годах для пенсионеров и безработных.

In [23]:
(df.query('income_type == "пенсионер" or income_type ==  "безработный"')['days_employed']/24/365).describe()

count    3445.000000
mean       41.667159
std         2.405824
min        37.526110
25%        39.570709
50%        41.691017
75%        43.749594
max        45.862489
Name: days_employed, dtype: float64

Средний стаж - 41 год. Это правдоподобные значения.

Выведем несколько значений для сверки, чтобы проверить значения в days_employed после коррекции.

In [24]:
df.loc[[5650, 14798]]

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,years_employed
5650,0,336658.500698,60.0,среднее,1,в разводе,3,F,пенсионер,0,169886.329146,операции с недвижимостью,922.352057
14798,0,395302.838654,45.0,Высшее,0,гражданский брак,1,F,безработный,0,202722.511368,ремонт жилью,1083.021476


Исправим данные для пенсионеров и безработных - переведем часы в дни и скорректируем новый столбец с годовым стажем.

In [25]:
df.loc[(df['income_type'] == 'пенсионер') | (df['income_type'] == 'безработный'), ['days_employed']] = ( df[(df['income_type'] == 'пенсионер') 
                                                                                                     | (df['income_type'] == 'безработный')]['days_employed']/24 )




df.loc[(df['income_type'] == 'пенсионер') | (df['income_type'] == 'безработный'), ['years_employed']] = ( df[(df['income_type'] == 'пенсионер') 
                                                                                                      | (df['income_type'] == 'безработный')]['days_employed']/365)

Проверим, что у нас получилось.

In [26]:
df.loc[[5650, 14798]]

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,years_employed
5650,0,14027.437529,60.0,среднее,1,в разводе,3,F,пенсионер,0,169886.329146,операции с недвижимостью,38.431336
14798,0,16470.951611,45.0,Высшее,0,гражданский брак,1,F,безработный,0,202722.511368,ремонт жилью,45.125895


Все корректно.

Проверим возраст пенсинеров и безработных.

In [27]:
df[(df['income_type'] == 'пенсионер') | (df['income_type'] == 'безработный')]['dob_years'].describe()

count    3858.000000
mean       59.276534
std         6.436018
min        22.000000
25%        56.000000
50%        60.000000
75%        64.000000
max        74.000000
Name: dob_years, dtype: float64

У нас есть молодые заемщики (минимальный возраст - 22 года) с большим трудовым стажем. Проверим это. Выведем такие строки, где трудовой стаж больше возраста заемщика.

In [28]:
outliners = df[df['years_employed']>= df['dob_years']]
display(outliners.head())
f'Процент заемщиков, у которых трудовой стаж больше возраста: {len(outliners)/df.shape[0]:.2%}'

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,years_employed
157,0,14517.251167,38.00000,среднее,1,женат / замужем,0,F,пенсионер,1,113560.650035,сделка с автомобилем,39.773291
578,0,16577.356876,43.29338,среднее,1,женат / замужем,0,F,пенсионер,0,97620.687042,строительство собственной недвижимости,45.417416
751,0,16281.477669,41.00000,среднее,1,женат / замужем,0,M,пенсионер,0,151898.693438,операции со своей недвижимостью,44.606788
776,0,15222.356680,38.00000,среднее,1,женат / замужем,0,F,пенсионер,0,73859.425084,покупка недвижимости,41.705087
1242,0,13948.510826,22.00000,Среднее,1,Не женат / не замужем,4,F,пенсионер,0,89368.600062,получение высшего образования,38.215098


'Процент заемщиков, у которых трудовой стаж больше возраста: 0.27%'

Это мизерный процент выбросов. Он не повлияет сильно на анализ.

#### Столбец gender

В столбце `gender` есть значение XNA.

In [29]:
f"Количество значений XNA в столбце с полом: {df[df['gender'] == 'XNA']['gender'].count()}"

'Количество значений XNA в столбце с полом: 1'

Всего лишь одно наблюдение -  можно безболезненно удалить строку из данных.

In [30]:
df = df.loc[df['gender'] != 'XNA']
f"Количество значений XNA в столбце с полом после удаления строки: {df[df['gender'] == 'XNA']['gender'].count()}"

'Количество значений XNA в столбце с полом после удаления строки: 0'

#### Столбец  family_status

Здесь нет артефактов, но исправим разный регистр. 

В [начале исследования](#intro) мы смотрели на уникальные значения данных для строковых столбцов в датафрейме и обнаружили неявные дубликаты в значениях столбцов 'family_status', 'eduaction'.


In [31]:
df['family_status'].value_counts()

женат / замужем          12380
гражданский брак          4176
Не женат / не замужем     2813
в разводе                 1195
вдовец / вдова             960
Name: family_status, dtype: int64

Устраним эту проблему с помощью метода str.lower() и проверим результат.

In [32]:
df['family_status'] = df['family_status'].str.lower()
df['family_status'].value_counts()

женат / замужем          12380
гражданский брак          4176
не женат / не замужем     2813
в разводе                 1195
вдовец / вдова             960
Name: family_status, dtype: int64

#### Столбец education

Приведем к нижнему регистру все значения в столбце education и тут же проверим результат выполнения метода str.lower().

In [33]:
df['education'] = df['education'].str.lower()
print(df['education'].value_counts())

среднее                15233
высшее                  5260
неоконченное высшее      743
начальное                282
ученая степень             6
Name: education, dtype: int64


### Обработка пропусков

[Ранее](#null_values) мы определили количество пропусков в столбцах `days_employed` и `total_income`.

Посмотрим на первые 5 строк датафрейма со строками с nan

In [34]:
df[df.isnull().any(axis='columns')].head()

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,years_employed
12,0,NaN,65.0,среднее,1,гражданский брак,1,M,пенсионер,0,NaN,сыграть свадьбу,NaN
26,0,NaN,41.0,среднее,1,женат / замужем,0,M,госслужащий,0,NaN,образование,NaN
29,0,NaN,63.0,среднее,1,не женат / не замужем,4,F,пенсионер,0,NaN,строительство жилой недвижимости,NaN
41,0,NaN,50.0,среднее,1,женат / замужем,0,F,госслужащий,0,NaN,сделка с подержанным автомобилем,NaN
55,0,NaN,54.0,среднее,1,гражданский брак,1,F,пенсионер,1,NaN,сыграть свадьбу,NaN


Проверим распределение пропусков в столбцах с доходом и стажем, действительно ли оно одинаковое:

In [35]:
len(df[df["total_income"].isnull() & df["days_employed"].isnull()])

2174

Пропуски в доходе и стаже распределены одинаково и носят неслучайный характер: если человек не работает, то и доход (по крайней мере официальный) не получает.

Посмотрим, как распределяются по типу занятости и полу заемщики с пропущенными данными о доходе и стаже.

In [36]:
df[df["total_income"].isnull() & df["days_employed"].isnull()].groupby('income_type')['gender'].value_counts()

income_type      gender
госслужащий      F         112
                 M          35
компаньон        F         327
                 M         181
пенсионер        F         352
                 M          61
предприниматель  M           1
сотрудник        F         693
                 M         412
Name: gender, dtype: int64

Это пенсионеры и, может быть, люди в декретном отпуске.

#### Обработка пропусков в столбце ежемесячного дохода (total_income) <a class="tocSkip">

Заменим пропуски в столбце ежемесячного дохода медианой, а не средним, поскольку у нас большой разброс значений в этом столбце.

Сгруппируем данные по типу занятости и посчитаем медиану ежемесячного дохода для каждого типа.

In [37]:
df.groupby('income_type')['total_income'].median()

income_type
безработный        131339.751676
в декрете           53829.130729
госслужащий        150447.935283
компаньон          172319.266339
пенсионер          118514.486412
предприниматель    499163.144947
сотрудник          142594.396847
студент             98201.625314
Name: total_income, dtype: float64

Выведем сначала 4 строки с пропусками в доходе и с разным типом занятости, чтобы потом проверить, корректно ли мы заполнили пропуски в столбце.

In [38]:
df.loc[[8963, 13716, 10876, 12338]]

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,years_employed
8963,0,NaN,53.0,высшее,0,гражданский брак,1,F,пенсионер,0,NaN,свой автомобиль,NaN
13716,0,NaN,26.0,неоконченное высшее,2,женат / замужем,0,F,сотрудник,0,NaN,ремонт жилью,NaN
10876,1,NaN,34.0,среднее,1,гражданский брак,1,F,компаньон,0,NaN,свадьба,NaN
12338,0,NaN,34.0,высшее,0,гражданский брак,1,M,госслужащий,0,NaN,сыграть свадьбу,NaN


Применим метод transform, чтобы на ходу посчитать медиану в сгруппированных данных, и заполним пропуски с помощью метода fillna.

In [39]:
df['total_income'] = df['total_income'].fillna(df.groupby('income_type')['total_income'].transform('median'))

Проверим, как заполнились пропуски в столбце income_type.

In [40]:
df.loc[[8963, 13716, 10876, 12338]]

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,years_employed
8963,0,NaN,53.0,высшее,0,гражданский брак,1,F,пенсионер,0,118514.486412,свой автомобиль,NaN
13716,0,NaN,26.0,неоконченное высшее,2,женат / замужем,0,F,сотрудник,0,142594.396847,ремонт жилью,NaN
10876,1,NaN,34.0,среднее,1,гражданский брак,1,F,компаньон,0,172319.266339,свадьба,NaN
12338,0,NaN,34.0,высшее,0,гражданский брак,1,M,госслужащий,0,150447.935283,сыграть свадьбу,NaN


Проверим, не осталось ли у нас пропусков в столбце income_type

In [41]:
df['total_income'].isna().sum()

0

#### Обработка пропусков в столбце трудового стажа (days_employed) <a class="tocSkip">

Заполним пропуски в столбце со стажем значением медианы в зависимости от возрастной группы клиента. 

##### Категоризация данных о возрасте заемщиков

Сначала распределим заемщиков по возрастным группам. Для этого создадим функцию для категоризации данных о возрасте клиентов.

In [42]:
# Функция для категоризации данных о возрасте клиентов

'''
Функция берет на вход возраст клиента - возвращает возрастную группу по значению возраста age, используя правила:
    - ранний возраст (19 - 30)
    - средний возраст (30 - 40)
    - поздний возраст (40 - 59)
    - пенсионный возраст (60 и старше)

'''

def get_age_group(age):
    if 30 > age >= 19:
        return 'ранний возраст'
    elif 40 > age >=30:
        return 'средний возраст' 
    elif 59 >= age >=40:
        return 'поздний возраст'
    else:
        return 'пенсионный возраст'
        

In [43]:
#Протестируем нашу функцию client_age_group на отдельном значении:
get_age_group(60)

'пенсионный возраст'

C помощью метода apply применим функцию get_age_group к столбцу `dob_years`, результат отбработки функции записываем в новый столбец 
`age_group`.

In [44]:
df['age_group'] = df['dob_years'].apply(get_age_group)

Посмотрим на результаты - выведем 5 случайных строк датафрейма.

In [45]:
df.sample(5)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,years_employed,age_group
16587,1,14965.817769,50.0,среднее,1,гражданский брак,1,F,пенсионер,0,115290.905764,операции с жильем,41.002240,поздний возраст
2438,0,14222.261144,57.0,среднее,1,женат / замужем,0,F,пенсионер,0,162490.548954,профильное образование,38.965099,поздний возраст
11347,0,2412.592209,26.0,высшее,0,гражданский брак,1,M,госслужащий,0,205568.828070,на проведение свадьбы,6.609842,ранний возраст
8504,0,5841.812213,46.0,среднее,1,гражданский брак,1,F,сотрудник,0,138665.848773,на проведение свадьбы,16.004965,поздний возраст
2640,2,2459.915170,35.0,среднее,1,женат / замужем,0,F,госслужащий,0,64001.658231,дополнительное образование,6.739494,средний возраст


Функция отработала корректно.

Выведем сначала 4 строки с пропусками в стаже и с разными возрастными категориями, чтобы потом проверить, корректно ли мы заполнили пропуски в столбце. 

In [46]:
df.loc[[12,26,21510,17242]]

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,years_employed,age_group
12,0,NaN,65.0,среднее,1,гражданский брак,1,M,пенсионер,0,118514.486412,сыграть свадьбу,NaN,пенсионный возраст
26,0,NaN,41.0,среднее,1,женат / замужем,0,M,госслужащий,0,150447.935283,образование,NaN,поздний возраст
21510,2,NaN,28.0,среднее,1,женат / замужем,0,F,сотрудник,0,142594.396847,приобретение автомобиля,NaN,ранний возраст
17242,1,NaN,32.0,среднее,1,гражданский брак,1,F,сотрудник,0,142594.396847,жилье,NaN,средний возраст


Более внимательно изучим данные о трудовом стаже в годах.

In [47]:
df['years_employed'].describe()

count    19350.000000
mean        12.717148
std         14.674185
min          0.066141
25%          2.539700
50%          6.011558
75%         15.173762
max         50.380685
Name: years_employed, dtype: float64

Разброс в данных большой - будем использовать медиану для заполнения пропусков в данном столбце в зависимости от возраста заемщика.

Выведем медиану стажа в днях и годах для каждой возрастной группы, чтобы потом проверить, корректно ли мы заполними пропуски.

In [48]:
df.groupby('age_group')['years_employed'].median()

age_group
пенсионный возраст    40.589871
поздний возраст        7.782670
ранний возраст         2.736882
средний возраст        4.388450
Name: years_employed, dtype: float64

In [49]:
df.groupby('age_group')['days_employed'].median()

age_group
пенсионный возраст    14815.302792
поздний возраст        2840.674560
ранний возраст          998.961907
средний возраст        1601.784231
Name: days_employed, dtype: float64

Сгруппируем данные по возрастным категориям, посчитаем медиану стажа в каждой и через метод transform подадим в метод fillna. 

Заполним таким образом пропуски в обоих столбцах со стажем - в днях и годах.

In [50]:
df['days_employed'] = df['days_employed'].fillna(df.groupby(['age_group'])['days_employed'].transform('median'))
df['years_employed'] = df['years_employed'].fillna(df.groupby(['age_group'])['years_employed'].transform('median'))

Проверим строки с пропусками по разным возрастным категориям, которые мы ранее отфильтровали.

In [51]:
df.loc[[12,26,21510,17242]]

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,years_employed,age_group
12,0,14815.302792,65.0,среднее,1,гражданский брак,1,M,пенсионер,0,118514.486412,сыграть свадьбу,40.589871,пенсионный возраст
26,0,2840.674560,41.0,среднее,1,женат / замужем,0,M,госслужащий,0,150447.935283,образование,7.782670,поздний возраст
21510,2,998.961907,28.0,среднее,1,женат / замужем,0,F,сотрудник,0,142594.396847,приобретение автомобиля,2.736882,ранний возраст
17242,1,1601.784231,32.0,среднее,1,гражданский брак,1,F,сотрудник,0,142594.396847,жилье,4.388450,средний возраст


Мы корректно заполнили пропуски в столбцах со стажем.

Подсчитаем пропуски в нашем датафрейме - проверим результаты.

In [52]:
df.isna().sum()

children            0
days_employed       0
dob_years           0
education           0
education_id        0
family_status       0
family_status_id    0
gender              0
income_type         0
debt                0
total_income        0
purpose             0
years_employed      0
age_group           0
dtype: int64

### Вывод

<div style="border:solid green 2px; padding: 20px">

* В ходе анализа выяснилось, что данные о трудовом стаже пенсионеров и безработных были представлены не в днях, а часах. Проблема устранена: эти значения переведены в дни.
    
* Для удобства анализа данных о стаже добавлен столбец `years_employed` (стаж в годах).
    
* Ежемесячный доход и трудовой стаж  — это количественные переменные. 
Пропуски в таких переменных, как правило, заполняют характерными значениями в выборке. Для типичных значений мы взяли **медиану**, потому что она позволяет избежать влияния выбросов в данных при расчетах. 

* Были категоризированы данные о возрасте заемщиков (добавлен столбец `age_group`). Выделены 4 группы заемщиков: пенсионный возраст (60 лет и старше) , поздний возраст (40 - 59 лет), средний возраст (30 - 40 лет), ранний возраст (19 - 30 лет). 
    
* Заполнены пропуски в столбце total_income значением медианы в зависимости от типа занятости. 
А в столбце трудового стажа - медианой в зависимости от возрастной группы.
    
* Стоит отметить, что пропущенные значения в трудовом стаже не влияют на результаты данного исследования, однако пропуски обработаны и в этом столбце, чтобы была возможность использовать данные для возможных дальнейших исследований. 
    
* Возможно, стоит обратить внимание разработчиков на проблему отрицательных значений в days_employed (предположительно, при подготовке и вычислении данных разработчики перепутали местами даты начала трудовой деятельности и текущий (или последний) день работы).

### Замена типа данных

Посмотрим на типы данных в нашем датафрейме.

In [53]:
df.dtypes

children              int64
days_employed       float64
dob_years           float64
education            object
education_id          int64
family_status        object
family_status_id      int64
gender               object
income_type          object
debt                  int64
total_income        float64
purpose              object
years_employed      float64
age_group            object
dtype: object

Заменим числа с плавающей точкой на целые для столбцов days_employed, dob_years, total_income 

с помощью метода astype и словаря, где ключ - название столбца, а значение - тип данных

In [54]:
df = df.astype({'days_employed': 'int64', 'years_employed': 'int64', 'dob_years': 'int64', 'total_income': 'int64'})
print('Типы данных в датафрейме после замены: \n')
print(df.dtypes)

Типы данных в датафрейме после замены: 

children             int64
days_employed        int64
dob_years            int64
education           object
education_id         int64
family_status       object
family_status_id     int64
gender              object
income_type         object
debt                 int64
total_income         int64
purpose             object
years_employed       int64
age_group           object
dtype: object


#### Вывод

<div style="border:solid green 2px; padding: 20px">
    
Дробные числа (float64) заменены на целые (int64) в столбцах:
    
* трудовой стаж в днях (days_employed), 
* трудовой стаж в годах (years_employed),    
* возраст клиентов (dob_years),
* ежемесячный доход (total_income) 

### Обработка дубликатов

Ранее мы удалили неявные дубликаты, сделав одинаковый - нижний - регистр в данных столбцов education, family_status.

Посмотрим теперь на количество явных дубликатов в нашем датафрейме с помощью цепочки методов duplicated().sum().

In [55]:
f'Количество явных дубликатов в таблице: {df.duplicated().sum()}'

'Количество явных дубликатов в таблице: 71'

Ожидаемо, что их стало больше. 

Удаляем явные дубликаты вместе с их индексами с помощью метода drop_duplicates() и reset_index(drop=True).

In [56]:
df = df.drop_duplicates().reset_index(drop=True)
f'Количество дубликатов после применения метода drop_duplicates: {df.duplicated().sum()}'

'Количество дубликатов после применения метода drop_duplicates: 0'

In [57]:
f'Количество наблюдений в данных после удаления всех дубликатов: {df.shape[0]}'

'Количество наблюдений в данных после удаления всех дубликатов: 21453'

#### Вывод

<div style="border:solid green 2px; padding: 20px">

В нашем датасете были обнаружены явные и неявные дубликаты. Причина появления неявных дубликатов: строковые значения набраны разными регистрами. Эта проблема устранена с помощью метода  str.lower().
    
Явные дубликаты успешно удалены с помощью цепочки методов drop_duplicates().reset_index(drop=True).

### Лемматизация данных о целях кредита

Чтобы корректно категоризовать данные о цели, прежде лемматизируем текст в столбце с целями. 

Посмотрим на значения в столбце с данными о цели получения кредита, чтобы понимать, что будем лемматизировать. 

In [58]:
df['purpose'].value_counts()

свадьба                                   791
на проведение свадьбы                     768
сыграть свадьбу                           765
операции с недвижимостью                  675
покупка коммерческой недвижимости         661
операции с жильем                         652
покупка жилья для сдачи                   651
операции с коммерческой недвижимостью     650
покупка жилья                             646
жилье                                     646
покупка жилья для семьи                   638
строительство собственной недвижимости    635
недвижимость                              633
операции со своей недвижимостью           627
строительство жилой недвижимости          624
покупка недвижимости                      620
покупка своего жилья                      620
строительство недвижимости                619
ремонт жилью                              607
покупка жилой недвижимости                606
на покупку своего автомобиля              505
заняться высшим образованием      

Получим леммы в значениях столбца с целями получения кредита

Создадим функцию, которая лемматизирует текст (то есть приводит слова в тексте к начальной форме слова),
а также удаляет пробелы и стоп-слова и возращает только полнозначные слова в начальной форме.

In [59]:
# Функция для лемматизации текста и удаления стоп-слов. 

"""
Аргумент функции text - тип данных строка.
Функция возращает строку из ключевых лемм - тип данных строка.

"""

#Возьмем список стоп-слов для русского языка из nltk.stopwords:
russian_stopwords = stopwords.words('russian')

def lemmatize_text(text):
    lemmas = mystem.lemmatize(text) #лемматизируем текст
    lemmas_only_key_words = [] #создаем пустой список для лемм полнозначных слов
    for lemma in lemmas:
#если лемма не является пробелом и не принадлежит к числу стоп-слов, то добавляем ее в наш список
        if lemma != ' ' and lemma not in russian_stopwords: 
            lemmas_only_key_words.append(lemma)
            
#склеиваем список в строку, разделяя леммы пробелом
    key_lemmas = ' '.join(lemmas_only_key_words).strip()
    return key_lemmas

Протестируем функцию на отдельном значении.

In [60]:
lemmatize_text('операции с коммерческой недвижимостью')

'операция коммерческий недвижимость'

Применим нашу функцию к столбцу с целями получения кредита (purpose) и запишем значения в новый столбец purpose_words.

In [61]:
df['purpose_words'] = df['purpose'].apply(lemmatize_text)
df.head()

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,years_employed,age_group,purpose_words
0,1,8437,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,покупка жилья,23,поздний возраст,покупка жилье
1,1,4024,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля,11,средний возраст,приобретение автомобиль
2,0,5623,33,среднее,1,женат / замужем,0,M,сотрудник,0,145885,покупка жилья,15,средний возраст,покупка жилье
3,3,4124,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628,дополнительное образование,11,средний возраст,дополнительный образование
4,0,14177,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьбу,38,поздний возраст,сыграть свадьба


Функция отработала корректно.

Теперь посмотрим на список получившихся лемм в столбце `purpose_words`.

In [62]:
df['purpose_words'].value_counts()

автомобиль                                972
свадьба                                   791
проведение свадьба                        768
сыграть свадьба                           765
операция недвижимость                     675
покупка коммерческий недвижимость         661
операция жилье                            652
покупка жилье сдача                       651
операция коммерческий недвижимость        650
покупка жилье                             646
жилье                                     646
покупка жилье семья                       638
строительство собственный недвижимость    635
недвижимость                              633
операция свой недвижимость                627
строительство жилой недвижимость          624
покупка недвижимость                      620
покупка свой жилье                        620
строительство недвижимость                619
ремонт жилье                              607
покупка жилой недвижимость                606
покупка свой автомобиль           

#### Вывод

<div style="border:solid green 2px; padding: 20px"> 
    
Мы лемматизировали значения в столбце `purpose` с целями получения кредита с помощью библиотеки pymystem3, NLTK и создали новый столбец `purpose_words`. 

На основе этого столбца мы сможем легко и корректно категоризовать данных о целях получения кредита. 

### Категоризация данных о целях кредита

Создадим функцию, которая возвращает нам категорию цели получения кредита. 

In [63]:
#Функция для возвращения цели получения кредита

'''
Функция берет на вход строковое значение, возвращает целевую категорию в зависимости от ключевого существительного.
Там, где возможно классифицировать вид недвижимости (по наличию слов "жилой" и "коммерческий"), мы классифицируем. 
Там где это невозможно - оставляем общий термин "недвижимость".

Аргументы функции: тип данных строка
Функция возвращает строку.

'''

def get_purpose_category (string):
    
    if 'автомобиль' in string:
        return 'автомобиль'
    
    elif 'свадьба' in string:
        return 'свадьба'
    
    elif 'недвижимость' in string and 'коммерческий' in string:
        return 'коммерческая недвижимость'
    
    elif 'жилой' in string and 'недвижимость' in string:
        return 'жилая недвижимость'
    
    elif 'жилье' in string:
        return 'жилая недвижимость'
    
    elif 'образование' in string:
        return 'образование'
    
    else:
        return 'недвижимость'
        

In [64]:
#Протестируем функцию на отдельных значениях:
print(get_purpose_category('покупка свой автомобиль'))
print(get_purpose_category('операция коммерческий недвижимость'))
print(get_purpose_category('покупка жилье сдача'))
print(get_purpose_category('покупка жилой недвижимость'))   
print(get_purpose_category('операция свой недвижимость'))
print(get_purpose_category('сыграть свадьба'))
print(get_purpose_category('получение дополнительный образование'))      

автомобиль
коммерческая недвижимость
жилая недвижимость
жилая недвижимость
недвижимость
свадьба
образование


Функция работает корректно. Теперь все готово для категоризации данных о целчх получения кредита.

Вычислим целевые категории по ключевым словам из столбца 'purpose' с помощью нашей функции get_purpose_category внутри apply() 
и запишем новые значения в столбец purpose_category.

In [65]:
df['purpose_category'] = df['purpose_words'].apply(get_purpose_category)

Проверим распределение значений в новом столбце purpose_category и узнаем, на что чаще всего берут кредиты. 

In [66]:
df['purpose_category'].value_counts()

жилая недвижимость           5690
автомобиль                   4306
образование                  4013
недвижимость                 3809
свадьба                      2324
коммерческая недвижимость    1311
Name: purpose_category, dtype: int64

Столбец c леммами ключевых слов для цели нам более не нужен - удалим его с помощью метода drop() и сохраним изменения в исходном датафрейме с помощью параметра inplace=True.

In [67]:
df.drop('purpose_words', axis=1, inplace=True)

In [68]:
df.head()

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,years_employed,age_group,purpose_category
0,1,8437,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,покупка жилья,23,поздний возраст,жилая недвижимость
1,1,4024,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля,11,средний возраст,автомобиль
2,0,5623,33,среднее,1,женат / замужем,0,M,сотрудник,0,145885,покупка жилья,15,средний возраст,жилая недвижимость
3,3,4124,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628,дополнительное образование,11,средний возраст,образование
4,0,14177,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьбу,38,поздний возраст,свадьба


#### Вывод

<div style="border:solid green 2px; padding: 20px"> 

Данные о целях получения кредита успешно категоризированы. 
    
Мы разделили все цели на 6 категорий: 

* жилая недвижимость
* коммерческая недвижимость
* недвижимость (общая категория для тех случаев, когда не было конкретизации)
* автомобиль
* образование 
* свадьба 

Можно утверждать, что чаще всего люди берут кредиты для вложения в жилую недвижимость, автомобиль и образование.
    
Этап предобработки данных завершен. Теперь проведем анализ очищенных данных и дадим ответы на вопросы кредитного отдела банка.

### Категоризация данных о доходе

[Ранее](#describe) мы выяснили, что в столбце с ежемесячным доходом большой разброс значений. Укрупним значения, чтобы было удобно дальше группировать заемщиков по количеству ежемесячного дохода, сравнивать каждую группу заемщиков и делать выводы.

"Разрежем" данные ежемесячного дохода на шесть интервалов: группа с максимально низким и группа с максимально высоким доходом (по 10% данных каждая), остальные группы с примерно равным количеством элементов. Границы относительно всех доходов: [10, 30, 50, 70, 90, 100].

Разделим данные столбца total_income на 6 интервалов (параметр q) с помощью метода qcut:

In [69]:
pd.qcut(df['total_income'], q=[0,.1,.3,.5,.7,.9,1]).value_counts()

(116007.4, 142594.0]     4407
(78719.4, 116007.4]      4290
(179797.2, 269829.0]     4290
(142594.0, 179797.2]     4174
(20666.999, 78719.4]     2146
(269829.0, 2265604.0]    2146
Name: total_income, dtype: int64

Создадим столбец `income_group` с интервалами ежемесячного дохода.

In [70]:
df['income_group'] = pd.qcut(df['total_income'], q=[0,.1,.3,.5,.7,.9,1])
df.head()

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,years_employed,age_group,purpose_category,income_group
0,1,8437,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,покупка жилья,23,поздний возраст,жилая недвижимость,"(179797.2, 269829.0]"
1,1,4024,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля,11,средний возраст,автомобиль,"(78719.4, 116007.4]"
2,0,5623,33,среднее,1,женат / замужем,0,M,сотрудник,0,145885,покупка жилья,15,средний возраст,жилая недвижимость,"(142594.0, 179797.2]"
3,3,4124,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628,дополнительное образование,11,средний возраст,образование,"(179797.2, 269829.0]"
4,0,14177,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьбу,38,поздний возраст,свадьба,"(142594.0, 179797.2]"


#### Вывод

<div style="border:solid green 2px; padding: 20px">  
    
Мы выделили 6 групп ежемесячного дохода (столбец `income_group`): 


| Ежемесячный доход (интервал)  | Количество заемщиков с доходом, в таком интервале   |
| ------- | -------- |
| 116007.4 - 142594.0   | 4407    |
| 78719.4 - 116007.4  | 4290    |
| 179797.2 - 269829.0  | 4290    |
| 142594.0 - 179797.2  | 4174    |
| 20666.999 - 78719.4  | 2146    |
| 269829.0 - 2265604.0  | 2146    |

## Ответы на вопросы кредитного отдела банка

#### Есть ли зависимость между наличием детей и возвратом кредита в срок?

Для ответа на вопрос сгруппируем заемщиков по количеству детей и вычислим отношение должников в каждой группе к общему количеству заемщиков в группе.

Это можно сделать с помощью функции среднего арифметического `mean`, так как у нас `debt` имеет значения 1 (есть задолжность) и 0 (задолжности нет).

In [71]:
pivot_children = df.pivot_table(index='children', values='debt', aggfunc='mean').rename(columns={'debt':'ratio_with_debts'})

# Отсортируем по столбцу с долей должников - от большего значения к меньшему.
pivot_children.sort_values(by='ratio_with_debts', ascending=False)

,ratio_with_debts
children,
20,0.105263
4,0.097561
2,0.094542
1,0.092346
3,0.081818
0,0.075263
5,0.000000


##### Вывод

<div style="border:solid green 2px; padding: 20px"> 

У заемщиков без детей процент задолжности ниже (7%), чем у клиентов с детьми ( 9 % - у клиентов с 1-2 и 4 детьми, 8% - у клиентов с 3 детьми).     
Наличие ребенка влияет негативным образом на факт погашения кредита в срок. Чаще всего кредит возращают в срок заемщики, у которых нет детей.

#### Есть ли зависимость между семейным положением и возвратом кредита в срок?

Сгруппируем заемщиков по семейному статусу и вычислим отношение должников в каждой группе к общему количеству заемщиков в группе.

In [72]:
pivot_family_status = df.pivot_table(index='family_status', values='debt', aggfunc='mean').rename(columns={'debt': 'ratio_with_debts'})
pivot_family_status.sort_values(by='ratio_with_debts', ascending=False)

,ratio_with_debts
family_status,
не женат / не замужем,0.097509
гражданский брак,0.093494
женат / замужем,0.075452
в разводе,0.071130
вдовец / вдова,0.065693


##### Вывод

<div style="border:solid green 2px; padding: 20px"> 
    
Чаще всего кредит возвращают в срок вдовцы (6% должников), разведенные (7% должников) и женатые и замужние (7% должников), а не замужние и не женатые (9% должников) и те, кто в гражданском браке (по 9 % должников), чаще всего имеют задолжность. 
    
Любопытно, что ответственность за принятие решения узаконить отношения как будто бы влияет на платежеспособность. 


#### Есть ли зависимость между уровнем дохода и возвратом кредита в срок?

Сгруппируем заемщиков по столбцу `income_group` и вычислим отношение должников в каждой группе к общему количеству заемщиков в группе.

In [73]:
pivot_income = df.pivot_table(index='income_group', values='debt', aggfunc='mean').rename(columns={'debt': 'ratio_with_debts'})
pivot_income.sort_values(by='ratio_with_debts', ascending=False)

,ratio_with_debts
income_group,
"(116007.4, 142594.0]",0.087815
"(78719.4, 116007.4]",0.085315
"(142594.0, 179797.2]",0.083852
"(179797.2, 269829.0]",0.076923
"(20666.999, 78719.4]",0.073159
"(269829.0, 2265604.0]",0.070363


##### Вывод
<div style="border:solid green 2px; padding: 20px"> 
    
Анализ данных в сводной таблице показал неожиданный результат: среди заемщиков с наименьшим доходом такой же процент должников (7%), как и среди заемщиков с самым высоким доходом. Обе эти категории заемщиков чаще всего возращают кредит в срок. Должников среди заемщиков с ежемесячным доходом в интервале от 78 719 до  179 797 выше на 1%.

#### Как разные цели кредита влияют на его возврат в срок?

Сгруппируем заемщиков по цели получения кредита и вычислим отношение должников в каждой группе к общему количеству заемщиков в группе.

In [74]:
pivot_purpose = df.pivot_table(index='purpose_category', values='debt', aggfunc='mean').rename(columns={'debt': 'ratio_with_debts'})
pivot_purpose.sort_values(by='ratio_with_debts', ascending=False)

,ratio_with_debts
purpose_category,
автомобиль,0.093590
образование,0.092200
свадьба,0.080034
коммерческая недвижимость,0.075515
недвижимость,0.075085
жилая недвижимость,0.069772


##### Вывод

<div style="border:solid green 2px; padding: 20px"> 
    
Чаще всего заемщики возвращают кредит в срок, если берут его с целью вложиться в недвижимость. Те, кто вкладывают в жилую недвижимость, - самые надежные плательщики и имеют самый низкий процент должников - 6%. А самыми ненадежными плательщиками являются заемщики, которые берут кредит на автомобиль и образование (9% должников в каждой категории). 

Может быть, кредиты на недвижимость кардинально меняют жизнь людей, потому что суммы существеннее и зачастую выплаты дольше. Это требует большой дисциплины.

В конце исследования еще раз посмотрим на наш датасет, чтобы убедиться, что он в нужном (очищенном и обогащенном) состоянии. 

In [75]:
df.head()

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,years_employed,age_group,purpose_category,income_group
0,1,8437,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,покупка жилья,23,поздний возраст,жилая недвижимость,"(179797.2, 269829.0]"
1,1,4024,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля,11,средний возраст,автомобиль,"(78719.4, 116007.4]"
2,0,5623,33,среднее,1,женат / замужем,0,M,сотрудник,0,145885,покупка жилья,15,средний возраст,жилая недвижимость,"(142594.0, 179797.2]"
3,3,4124,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628,дополнительное образование,11,средний возраст,образование,"(179797.2, 269829.0]"
4,0,14177,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьбу,38,поздний возраст,свадьба,"(142594.0, 179797.2]"


In [76]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21453 entries, 0 to 21452
Data columns (total 16 columns):
 #   Column            Non-Null Count  Dtype   
---  ------            --------------  -----   
 0   children          21453 non-null  int64   
 1   days_employed     21453 non-null  int64   
 2   dob_years         21453 non-null  int64   
 3   education         21453 non-null  object  
 4   education_id      21453 non-null  int64   
 5   family_status     21453 non-null  object  
 6   family_status_id  21453 non-null  int64   
 7   gender            21453 non-null  object  
 8   income_type       21453 non-null  object  
 9   debt              21453 non-null  int64   
 10  total_income      21453 non-null  int64   
 11  purpose           21453 non-null  object  
 12  years_employed    21453 non-null  int64   
 13  age_group         21453 non-null  object  
 14  purpose_category  21453 non-null  object  
 15  income_group      21453 non-null  category
dtypes: category(1), int64(

Сохраним датасет в таком виде для возможных дальнейших исследований.

In [77]:
df.to_csv('bank_clients_mortgage_clean.csv')

Для создания сводных таблиц в последнем разделе можно было создать свою функцию, но мы посчитали, что так расчеты будут более прозрачными для читающего.  

# Спасибо за внимание!